*Candidate Name: Joseph Thomas | Candidate Number: <redacted>*

CE310 Evolutionary Computation and Genetic Programming assignment

### Part 1.1:

Design and implement the core functions needed for a Steady-State binary GA.
Select and implement the following operators where appropriate for the problems to be solved. 
__Crossover__: Cutting two bit strings at a random position and swapping the left hand sides
__Mutation__: Bit Flip Mutation: randomly changing bit values 
__Selection__: Use the best X individuals for mating.

Select appropriate standard values for the hyper-parameters.




In [282]:
""" initial setup - taken from the unit 3 script """

import random

# Hyper-parameters
no_pop = 1000 # population size
no_sel = 6    # select the best no_sel individuals
no_gen = 12   # number of generations to run for
no_bit = 85   # size of the strings generated 
p_xo = .8     # crossover probabilty/rate
p_m = 0.04    # mutation probabilty/rate per bit

ascii = (
    "00011110000011000"
    "00100001000101100"
    "01000000001000110"
    "00100111001111110"
    "00011101001000010"
)

""" Functions """

'''
old fitness function that was useless...
with this function we would also change the index to ascending order
as the smaller the 'temp' (fitness) value the more fit the individual
'''

#def fitness(chromosome):
#    fit = []
#    for i in range(len(chromosome)):
#        temp = decimal(chromosome[i]) - decimal(ascii)
#        if temp < 0: temp = temp * -1
#        fit.append(temp)
            
#    return fit


# New fitness, checks how many bits match
def fitness(population):
    fit = []
    for i in range(len(population)):
        temp = 0
        for j in range(len(population[i])):
            if population[i][j] == ascii[j]:
                temp+=1
        fit.append(temp)
        
    return fit
        

# Same code as that provided in '222 problem'
def one_point_crossover(chromosome_1, chromosome_2):
    cut = random.randint(1, no_bit-1) 
    offspring = chromosome_1[0:cut] + chromosome_2[cut:]
    
    return offspring


# Randomly flips bits within an offspring
def bit_flip_mutation(chromosome):
    output = ""
    for i in range(len(chromosome)):
        if random.random() < p_m:
            if chromosome[i] == '0': temp = '1' 
            if chromosome[i] == '1': temp = '0' 
        else:
            temp = chromosome[i]
        output = output + temp
        
    return output


# Generates the population, code from '222 problem'
def generate_population(no_bit):
    pop = [] # population
    for i in range(no_pop):
        pop.append(''.join(random.choice('01') for j in range(no_bit)))
        
    return pop
        

# prints lists/array values on separate lines
def output(text):
    for i in range(len(text)):
        print(text[i])


# splits the string into 5 arrays of length 17, easier to
# print the ascii art from this.
def draw_art(text):
    # Easy string split into array, code taken from:
    # https://stackoverflow.com/questions/9475241/split-string-every-nth-character
    n = 17
    temp = [text[i:i+n] for i in range(0, len(text), n)]
    return temp
    
            

""" Main program """

pop = generate_population(no_bit)


for g in range(1, no_gen+1):
    fit = fitness(pop)
   
    index = sorted(range(len(fit)), key=lambda k: fit[k], reverse=True)
    
    pop_new = []
    for i in index[:no_sel]:
        pop_new.append(pop[i])
    
    for i in range(no_sel, no_pop):
        if random.random() <= p_xo:
            parent_1 = random.randint(0, no_sel-1)
            parent_2 = random.randint(0, no_sel-1)
            while parent_1 == parent_2:
                parent_2 = random.randint(0, no_sel-1) 
            offspring = one_point_crossover(pop_new[parent_1], pop_new[parent_2])
            
        else:
            offspring = pop_new[random.randint(0, no_sel-1)]
        
        offspring = bit_flip_mutation(offspring)
        pop_new.append(offspring)
        
    pop = pop_new
    
    
print("\nAscii:")
print(ascii)
print("\nMost fit:")
print(pop[index[0]])
print("\nFitness:")
print(fit[index[0]])
print("\nAscii art:")
output(draw_art(pop[index[0]]))

    




Ascii:
0001111000001100000100001000101100010000000010001100010011100111111000011101001000010

Most fit:
0001111000001100000100001000101100010000000010001100010011100111111000011101001000010

Fitness:
85

Ascii art:
00011110000011000
00100001000101100
01000000001000110
00100111001111110
00011101001000010


### Part 1.2: Hyper-Parameter analysis

___Overview of the program___:
The majority of the code is sampled from the '222 problem' example within Unit 3, with alterations to the fitness function, the bit flip mutation and the addition of probability crossover.

Originally I attempted to determine fitness as sum(chromosome) - sum(ascii), where the lower the value returned the more fit the individual, however was finding that whilst it would solve the latter 80-90% of the problem quite easily, it would never get the first couple of bits of the string correct. This fitness function also required running the code with a high population count for alot of generations to return the previously mentioned result, so I decided to try a different fitness function.

The new fitness function simply checks how many bits in the chromosome match that of the ascii string. The greatest fitness value is 85, where all characters match, and so those with the highest fitness score parent the next generation. With this method, we still maintain a larger population size, however require ___a lot___ less generations, 12 instead of 3000. 12 is just the number of generations I found to be most frequently accurate, along with selecting the 6 most fit individuals to parent the next generation. The other benefit to this fitness function is my bit flip mutation rate is also alot lower, at 0.04 opposed to the 0.1 required with my previous method, which is in part why it can reach a more accurate solution in less generations. My crossover rate is 0.8, as it manages to reach the correct solution in a smaller number of generations. By applying crossover so frequently over such a large population we stand more chance of positive mutations.

Upwards of 95% of the time the most fit individual(s) match the ascii value, however will very occasionally get a single bit incorrect. 

A drawback of my solution is it might not work with the hyper-parameter values used if applied to a different problem, as it is quite specific to this 85-bit string example. Theoretically, you wouldn't have to alter the code for a similar problem, just the hyper-parameter values.

___Selection___: 
The selection sample is very small in comparison to the population size to maintain high quality. I trialled values ranging from 5-50 and found 6 to be the most frequently accurate in correlation with my population size and generations.

___One Point Crossover___:
The one point crossover function behaves exactly as that from the 222 example, randomly dissecting two parents and splicing them to create new offspring. My crossover rate is 0.8 to allow frequent crossover in the hopes of positive mutation

___Bit Flip Mutation___:
I've found it best to keep the chance of a bit being flipped around the 0.05 value, we want this to be fairly low so that we aren't mistakenly flipping correct bits, but high enough that the program is stochastic and doesn't get stuck on an incorrect answer.



### Part 1.3: Fitness Function:

In [6]:
import math
# Rastrigin function
# f(x) = A(n) + sum(1 to n)[x** - Acos(2pi(x))]
A = 10
n = 5
fx = A*n
for i in range(1, n+1):
    fx = fx + (i**2)-(10*math.cos(2*math.pi*i))
print(fx)

# 
x = 1
y = 1
fxy = (1-x)**2+100*(y-x**2)**2
print(fxy)

55.0
0


The rastrigin function attempts to find the global minimum, which can be difficult due to local minima. It searches within the domain -5.12 <= x <= 5.12, with 0 being the point of global minimum.

Rosenbrock constrained to a disk is also optimised my minimalising. Operates in the constaint -1.5 <= x <= 1.5. Convergence to the global minima can be difficult, as it lies within a parabolic valley.

### Part 1.4: Mini Project

In [352]:
""" Pattern detector """

import random

# Hyper-parameters
no_pop = 1000 # population size
no_sel = 5    # select the best no_sel individuals
no_gen = 11   # number of generations to run for
no_bit = 16   # size of the strings generated 
p_xo = .8     # crossover probabilty/rate
p_m = 0.04    # mutation probabilty/rate per bit
filter = '0001'

""" Functions """


# New fitness, checks how many bits match
def fitness(population):
    fit = []
    for i in range(len(population)):
        count = 0
        for j in range(13):
            temp = population[i][j]+population[i][j+1]+population[i][j+2]+population[i][j+3]
            if temp == filter: count+=1
        fit.append(count)
    
    return fit
        

# Same code as that provided in '222 problem'
def one_point_crossover(chromosome_1, chromosome_2):
    cut = random.randint(1, no_bit-1) 
    offspring = chromosome_1[0:cut] + chromosome_2[cut:]
    
    return offspring


# Randomly flips bits within an offspring
def bit_flip_mutation(chromosome):
    output = ""
    for i in range(len(chromosome)):
        if random.random() < p_m:
            if chromosome[i] == '0': temp = '1' 
            if chromosome[i] == '1': temp = '0' 
        else:
            temp = chromosome[i]
        output = output + temp
        
    return output


# Generates the population, code from '222 problem'
def generate_population(no_bit):
    pop = [] # population
    for i in range(no_pop):
        pop.append(''.join(random.choice('01') for j in range(no_bit)))
        
    return pop
        

# prints lists/array values on separate lines
def output(text):
    for i in range(len(text)):
        print(text[i])


# splits the string into 5 arrays of length 17, easier to
# print the ascii art from this.
def draw_art(text):
    # Easy string split into array, code taken from:
    # https://stackoverflow.com/questions/9475241/split-string-every-nth-character
    n = 17
    temp = [text[i:i+n] for i in range(0, len(text), n)]
    return temp
   

""" Main program """

pop = generate_population(no_bit)


for g in range(1, no_gen+1):
    fit = fitness(pop)
   
    index = sorted(range(len(fit)), key=lambda k: fit[k], reverse=True)
    
    pop_new = []
    for i in index[:no_sel]:
        pop_new.append(pop[i])
    
    for i in range(no_sel, no_pop):
        if random.random() <= p_xo:
            parent_1 = random.randint(0, no_sel-1)
            parent_2 = random.randint(0, no_sel-1)
            while parent_1 == parent_2:
                parent_2 = random.randint(0, no_sel-1) 
            offspring = one_point_crossover(pop_new[parent_1], pop_new[parent_2])
            
        else:
            offspring = pop_new[random.randint(0, no_sel-1)]
        
        offspring = bit_flip_mutation(offspring)
        pop_new.append(offspring)
        
    pop = pop_new
    
print("\nMost fit:")
print(pop[index[0]])
print("\nFitness:")
print(fit[index[0]])


Most fit:
0001000100010001

Fitness:
4


My program finds 4-bit patterns specified by the user (within the hyper-parameters) within 16 bit binary strings. The hyper-parameters are rather similar to that from 1.1, yeilding the best performance in the fewest number of generations. The fitness function works as a sliding kernel, checking all 4-bit combinations (13 in total) within the binary string, and whenever the pattern is found, increments the counter. The more times the pattern is found, the higher the fitness of the chromosome. 

### Self assessment

__Part 1.1/1.2__:
I am happy with my GA, with the given hyper parameters it appears to find the correct solution

__Part 1.3__:
Really not sure what the brief intended unfortunately, have implemented the code as requested with a brief analysis of what is supposed to be happening.

__Part 1.4__:
I am happy with my personal project, whilst it is similar in functionality to task 1.1, it is unique enough to require a different fitness function and hyper parameters, and appears to find correct solutions. I wanted to make it function for 32 bit strings, and with enough alterations to hyper parameters I believe it would, but don't want to take up too much time computing the result.